## simple model Prediction 

In [1]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import time
from rembg import remove 
from PIL import Image, ImageFont, ImageDraw

# Initialize webcam
webcam = cv2.VideoCapture(0)

# Define the region of interest
region_top = 40
region_bottom = 314
region_left = 10
region_right = 314

# Load your pre-trained model
model2 = load_model("newdata.h5")

# Define class labels
class_labels =  [ 'ક','ખ', 'ગ','ઘ', 'ચ', 'છ', 'જ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન',
     'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'વ','શ','ષ' , 'સ' ,'હ','ળ','ક્ષ', 'જ્ઞ','0']
# Load Gujarati font
font_path = "NotoSerifGujarati-Regular.ttf"
font = ImageFont.truetype(font_path, 32)

# Preprocess image for model prediction
def preprocess_image(image_path):
    img_ = cv2.imread(image_path, cv2.IMREAD_ANYCOLOR)
    img_ = cv2.fastNlMeansDenoisingColored(img_, None, 3, 11, 7, 21)  # Denoise image
    img_ = cv2.resize(img_, (224, 224))  # Resize to match model's input shape
    img_array = img_to_array(img_)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Classify image and get predicted label
def classify_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model2.predict(img_array)
    # predicted_class = np.argmax(prediction)
    # predicted_label = class_labels[predicted_class]  
    return prediction

def print_label(prediction):
    dic = {}
    for i in range(len(class_labels)):
        dic[class_labels[i]] = prediction[0][i]
    out =list( sorted(dic.items(), key=lambda x: x[1],reverse=True))
    return out[:5]    

# Initialize time for label updates
last_pred_time = time.time()
predicted_label= list((list((" "," ")) , list((" ", " ")),))

while True:
    try:
        check, frame = webcam.read()
        
        if not check:
            break
        
        # Define the region of interest (ROI) and draw rectangle
        cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (0, 0, 0), 2)

        # Capture the region of interest (ROI)
        region = frame[region_top:region_bottom, region_left:region_right]
        
        if time.time() - last_pred_time >= 5:
            cv2.imwrite(filename='saved_img.jpg', img=region)
            input_img = Image.open("saved_img.jpg")
            output_img = remove(input_img)
            output_img.save("saved_img2.png", "PNG")
            prediction = classify_image('saved_img2.png')
            predicted_label = print_label(prediction)

            last_pred_time = time.time()

        # Convert OpenCV image to PIL for text rendering
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)

        # Render Gujarati text with PIL
        draw.text((region_left, region_top - 35), f"અનુમાનિત અક્ષર  : {predicted_label[0][0]} {predicted_label[1][0]}", font=font, fill=(0, 0, 0))
        # draw.text((region_left, region_bottom + 10), f"Predicted: {predicted_label2}", font=font, fill=(0, 0, 0))

        # Convert back to OpenCV format
        frame_with_text = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        # Show the frame with prediction
        cv2.imshow("Capturing", frame_with_text)
 
        # Check if 'q' is pressed to exit
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    except KeyboardInterrupt:
        print("Turning off camera.")
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 392ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import time
from rembg import remove 
from PIL import Image, ImageFont, ImageDraw
import pyttsx3


# Initialize webcam
webcam = cv2.VideoCapture(0)

# Define the region of interest
region_top = 40
region_bottom = 314
region_left = 10
region_right = 314

# Load your pre-trained model
model2 = load_model("newdata.h5")

# Define class labels
class_labels =  [ 'ક','ખ', 'ગ','ઘ', 'ચ', 'છ', 'જ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન',
     'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'વ','શ','ષ' , 'સ' ,'હ','ળ','ક્ષ', 'જ્ઞ','0']
transliterated_labels = ['ka', 'kha', 'ga', 'gha', 'cha', 'chha', 'ja', 'ta', 'tha', 'da', 'dha', 'ana', 'ta', 'tha', 'da', 'dha', 'na',
                         'pha', 'ba', 'bha', 'ma', 'ya', 'ra', 'la', 'va', 'sha', 'sh', 'sa', 'ha', 'ala', 'khsa', 'gna', '0']
# Load Gujarati font
font_path = "NotoSerifGujarati-Regular.ttf"
font = ImageFont.truetype(font_path, 32)

# Preprocess image for model prediction
def preprocess_image(image_path):
    img_ = image_path
    img_ = cv2.fastNlMeansDenoisingColored(img_, None, 3, 11, 7, 21)  # Denoise image
    img_ = cv2.resize(img_, (224, 224))  # Resize to match model's input shape
    img_array = img_to_array(img_)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Classify image and get predicted label
def classify_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model2.predict(img_array)
    predicted_class = np.argmax(prediction)
    predicted_label = class_labels[predicted_class]  
    trans_label = transliterated_labels[predicted_class]
    return [predicted_label, trans_label]
  

def text_to_speech(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# Initialize time for label updates
last_pred_time = time.time()
prediction_label=" "
while True:
    try:
        check, frame = webcam.read()
        
        if not check:
            break
        
        # Define the region of interest (ROI) and draw rectangle
        cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (0, 0, 0), 2)

        # Capture the region of interest (ROI)
        region = frame[region_top:region_bottom, region_left:region_right]
        
        if time.time() - last_pred_time >= 5:
            prediction_label , trans_label= classify_image(region)
            text_to_speech(trans_label)

            last_pred_time = time.time()

        # Convert OpenCV image to PIL for text rendering
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)

        # Render Gujarati text with PIL
        draw.text((region_left, region_top - 35), f"અનુમાનિત અક્ષર  : {prediction_label} ", font=font, fill=(0, 0, 0))
        # draw.text((region_left, region_bottom + 10), f"Predicted: {predicted_label2}", font=font, fill=(0, 0, 0))

        # Convert back to OpenCV format
        frame_with_text = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        # Show the frame with prediction
        cv2.imshow("Capturing", frame_with_text)
    
        # Check if 'q' is pressed to exit
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    except KeyboardInterrupt:
        print("Turning off camera.")
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 831ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


## Model Prediction On -> model_on_Edge_RGB

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import time
from rembg import remove 
from PIL import Image, ImageFont, ImageDraw,ImageFilter
import tensorflow as tf 

# Initialize webcam
webcam = cv2.VideoCapture(0)

# Define the region of interest
region_top = 40
region_bottom = 314
region_left = 10
region_right = 314

# Load your pre-trained model
# model2 = tf.keras.models.load_model("model_on_Edge_RGB.h5",compile=True)
model2 = tf.keras.models.load_model("model_on_Edge_RGB.h5", compile=False)  
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Define class labels
class_labels =  [ 'ક','ખ', 'ગ','ઘ', 'ચ', 'છ', 'જ', 'ટ', 'ઠ', 'ડ', 'ઢ', 'ણ', 'ત', 'થ', 'દ', 'ધ', 'ન',
     'ફ', 'બ', 'ભ', 'મ', 'ય', 'ર', 'લ', 'વ','શ','ષ' , 'સ' ,'હ','ળ','ક્ષ', 'જ્ઞ','0']
# Load Gujarati font
font_path = "NotoSerifGujarati-Regular.ttf"
font = ImageFont.truetype(font_path, 32)

# Preprocess image for model prediction
def preprocess_image(image_path):
    img_ = cv2.imread(image_path, cv2.IMREAD_ANYCOLOR)
    # img_ = cv2.fastNlMeansDenoisingColored(img_, None, 3, 11, 7, 21)  # Denoise image
    img_ = cv2.resize(img_, (224, 224))  # Resize to match model's input shape
    img_ = Image.fromarray(cv2.cvtColor(img_, cv2.COLOR_BGR2RGB))
    img_= img_.filter(ImageFilter.FIND_EDGES)
    img_ = img_.point(lambda x : 255 if x >45 else 0)   
    img_= img_.convert("RGB")   
    img_array = np.array(img_)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Classify image and get predicted label
def classify_image(image_path):
    img_array = preprocess_image(image_path)
    print(img_array.shape)
    prediction = model2.predict(img_array)
    # predicted_class = np.argmax(prediction)
    # predicted_label = class_labels[predicted_class]  
    return prediction


def print_label(prediction):
    dic = {}
    for i in range(len(class_labels)):
        dic[class_labels[i]] = prediction[0][i]
    out =list( sorted(dic.items(), key=lambda x: x[1],reverse=True))
    return out[:5]    

# Initialize time for label updates
last_pred_time = time.time()
predicted_label= list((list((" "," ")) , list((" ", " ")),))

while True:
    try:
        check, frame = webcam.read()
        
        if not check:
            break
        
        # Define the region of interest (ROI) and draw rectangle
        cv2.rectangle(frame, (region_left, region_top), (region_right, region_bottom), (0, 0, 0), 2)

        # Capture the region of interest (ROI)
        region = frame[region_top:region_bottom, region_left:region_right]
        
        if time.time() - last_pred_time >= 5:
            cv2.imwrite(filename='saved_img.jpg', img=region)
            input_img = Image.open("saved_img.jpg")
            # output_img = remove(input_img)
            # output_img.save("saved_img2.png", "PNG")
            prediction = classify_image('saved_img.jpg')
            predicted_label = print_label(prediction)

            last_pred_time = time.time()

        # Convert OpenCV image to PIL for text rendering
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(pil_image)

        # Render Gujarati text with PIL
        draw.text((region_left, region_top - 35), f"અનુમાનિત અક્ષર  : {predicted_label[0][0]} {predicted_label[1][0]}", font=font, fill=(0, 0, 0))
        # draw.text((region_left, region_bottom + 10), f"Predicted: {predicted_label2}", font=font, fill=(0, 0, 0))

        # Convert back to OpenCV format
        frame_with_text = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        # Show the frame with prediction
        cv2.imshow("Capturing", frame_with_text)
 
        # Check if 'q' is pressed to exit
        key = cv2.waitKey(1)
        if key == ord('q'):
            break

    except KeyboardInterrupt:
        print("Turning off camera.")
        break

# Release resources
webcam.release()
cv2.destroyAllWindows()


(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 659ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 250ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 331ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
(1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340

In [18]:
webcam.release()
cv2.destroyAllWindows()

In [12]:
preprocess_image("saved_img.jpg").shape

(224, 224, 3)

In [8]:
from google.cloud import texttospeech
import os

# Set your Google Cloud credentials (replace with your path)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/your/google-cloud-credentials.json"

def synthesize_gujarati_google(text):
    client = texttospeech.TextToSpeechClient()
    input_text = text_to_speech.SynthesisInput(text=text)

    # Select the Gujarati voice
    voice = text_to_speech.VoiceSelectionParams(
        language_code="gu-IN",
        name="gu-IN-Wavenet-A",  # Or gu-IN-Standard-A
        ssml_gender=text_to_speech.SsmlVoiceGender.FEMALE # Or MALE
    )

    audio_config = text_to_speech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )

    response = client.synthesize_speech(
        input=input_text, voice=voice, audio_config=audio_config
    )

    # The response's audio_content is binary. You can save it to a file
    # and then play it using a library like `playsound` or `pydub`.
    with open("output.mp3", "wb") as out:
        out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')

    # To play the audio (requires playsound library: pip install playsound)
    # from playsound import playsound
    # playsound("output.mp3")

gujarati_text = "નમસ્તે! આ એક ટેક્સ્ટ ટુ સ્પીચ પરીક્ષણ છે."
synthesize_gujarati_google(gujarati_text)

ImportError: cannot import name 'texttospeech' from 'google.cloud' (unknown location)